In [1]:
import json
from pathlib import Path
import pandas as pd
import os

RESULTS_DIR = Path("evaluation_results")

In [2]:
# Aggregate results
results = []
for f in os.listdir(RESULTS_DIR):
    with open(RESULTS_DIR / f, "r") as f:
        results.append(json.load(f))
        
# Save and print results
results_df = pd.DataFrame(results)

In [3]:
results_df['cot_correct'] = results_df['cot_answer'] == results_df['gold_answer']
results_df['finetuned_correct'] = results_df['finetuned_answer'] == results_df['gold_answer']
cot_accuracy = results_df['cot_correct'].mean()
finetuned_accuracy = results_df['finetuned_correct'].mean()
print(f"CoT accuracy: {cot_accuracy}")
print(f"Finetuned accuracy: {finetuned_accuracy}")

CoT accuracy: 0.8051554207733131
Finetuned accuracy: 0.8036391205458681


In [4]:
# see rows where critic was invoked
dfc = results_df[results_df['finetuned_output'].str.contains('<|endofturn|><|critic|>', regex=False)]
dfc

,question,gold_answer,cot_output,cot_answer,finetuned_output,finetuned_answer,cot_correct,finetuned_correct
54,"For a New Year's resolution, Andy wants to los...",525,To determine the total number of days Andy has...,None,<|begin_of_text|><|begin_of_text|><|start_head...,523,False,False
371,Cole wanted to buy new jeans for a dance conte...,8,"Step 1: First, let's calculate the original pr...",4,<|begin_of_text|><|begin_of_text|><|start_head...,None,False,False
471,Each solid 10-foot section of a redwood tree w...,5600,"To find the total weight of the redwood, we ne...",5600,<|begin_of_text|><|begin_of_text|><|start_head...,8000,True,False
481,I have 10 liters of orange drink that are two-...,15,To find out how much water is in the remaining...,None,<|begin_of_text|><|begin_of_text|><|start_head...,58,False,False
492,James gets 10 new CDs. Each CD cost $15. He ...,50,To find the total amount James spent on the CD...,None,<|begin_of_text|><|begin_of_text|><|start_head...,110,False,False
733,Manny signed up for Karate classes for $60. Hi...,4,Let's break it down step by step:\n\n1. The to...,9,<|begin_of_text|><|begin_of_text|><|start_head...,1,False,False
738,Juan and his brother Carlos are selling lemona...,15,"To solve this problem, we need to follow these...",15,<|begin_of_text|><|begin_of_text|><|start_head...,6,True,False
813,A custodian has to clean a school with 80 clas...,50,To find the total number of classrooms that ne...,None,<|begin_of_text|><|begin_of_text|><|start_head...,90,False,False
980,"I am three years younger than my brother, and ...",13,"To solve this problem, let's break it down ste...",13,<|begin_of_text|><|begin_of_text|><|start_head...,None,True,False
1119,John brings his dog to the vet. His dog needs ...,25,"To find out how much John leaves with, we need...",25,<|begin_of_text|><|begin_of_text|><|start_head...,25,True,True


In [5]:
s = dfc['finetuned_output'].iloc[4]
segs = s[s.find('<|solver|>', s.find('<|solver|>') + 1):].split('<|endofturn|>')[:-1]
print(s)

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a debate assistant. Respond with alternating segments that start with <|solver|> or <|critic|> and finish each turn with <|endofturn|>. Use no other tags.<|eot_id|><|start_header_id|>user<|end_header_id|>

<|question|> James gets 10 new CDs.  Each CD cost $15.  He gets them for 40% off.  He decides he doesn't like 5 of them and sells them for 40. How much money was he out?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|solver|> To find out how much James spent on the CDs, we need to calculate the original price and then subtract the discount.

Original price: 10 CDs x $15/CD = $150
Discount: 40% of $150 = 0.4 x $150 = $60
Price after discount: $150 - $60 = $90

James sold 5 CDs for $40 each, so he got a total of:
5 CDs x $40/CD = $200

Now, let's calculate how much money James was out:

Total amount spent: $90 (on the CDs) + $20

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

ADAPTER_DIR = Path("lora_debate_ckpts") / "checkpoint-600"  # last SFT ckpt
BASE_MODEL = "meta-llama/Llama-3.1-8B-Instruct"

def load_finetuned_model():
    """Load base model + LoRA adapters"""
    model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL, torch_dtype=torch.bfloat16, device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(ADAPTER_DIR)
    model.resize_token_embeddings(len(tokenizer))
    model = PeftModel.from_pretrained(model, ADAPTER_DIR)
    return model

In [8]:
model = load_finetuned_model()

Loading checkpoint shards: 100%|██████████| 4/4 [00:25<00:00,  6.40s/it]
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.96 GiB. GPU 0 has a total capacity of 14.57 GiB of which 1.31 GiB is free. Including non-PyTorch memory, this process has 13.26 GiB memory in use. Of the allocated memory 11.92 GiB is allocated by PyTorch, and 1.18 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)